# Prerequisite

Following packages must be installed to run codes below

```bash
$ pip3 install openpyxl 
```


In [1]:
%load_ext autoreload
%autoreload 2

# Add project path

In [2]:
import os
import sys
from pathlib import Path

PROJECT_PATH = Path().resolve().parent.parent
SRC_PATH = PROJECT_PATH / 'src'

sys.path.append(str(SRC_PATH))

In [3]:
exp_name = 'feature_selection_v4.0'

In [4]:
from project_paths import DATA_PATH
import pandas as pd

EXP_RESULT_PATH = DATA_PATH / 'experiment_result' / 'feature_selection_v2'

aggregated_result_path = EXP_RESULT_PATH / 'result.csv'

v2_exp_result = pd.read_csv(aggregated_result_path)

In [5]:
sectors = [
    '제조업(10~34)',
    '부동산업(68)',
    '도매 및 소매업(45~47)',
    '숙박 및 음식점업(55~56)',
    '건설업(41~42)'
]
years_to_close = [
    'Closed_In_1Yr',
    'Closed_In_2Yrs',
]

In [6]:
best_features = {}

for sector in sectors : 
    best_features[sector] = {}

for sector in sectors : 
    for target_years_to_close in years_to_close : 
        best_features[sector][target_years_to_close] = (
            v2_exp_result
            .loc[lambda x : x['대분류']==sector]
            .loc[lambda x : x['Fold']=='Mean']
            .loc[lambda x : x['target']==target_years_to_close]
            .set_index('feature_selection')
            ['AUC'].idxmax()
        )

In [7]:
from pprint import pprint
pprint(best_features)

{'건설업(41~42)': {'Closed_In_1Yr': "['순운전자본/자산총계', '유동부채/유동자산', '유형자산', '자본금', "
                                 "'유보액/총자산(%)', '유보액/납입자본(%)', '차입금의존도(%)', "
                                 "'총자본회전율(회)', '매출채권회전율(회)', '매입채무회전율(회)', "
                                 "'Closed_In_1Yr']",
                'Closed_In_2Yrs': "['매출채권', '비유동자산', '유형자산', '판매비와관리비', "
                                  "'기업순이익률(%)', '매출액총이익률(%)', '경상수지비율', "
                                  "'차입금의존도(%)', '유동부채비율(%)', '미수금', "
                                  "'Closed_In_2Yrs']"},
 '도매 및 소매업(45~47)': {'Closed_In_1Yr': "['유동자산/부채총계', '순운전자본/자산총계', '유동자산', "
                                      "'유형자산', '자산총계', '유동부채', '이익잉여금(결손금）', "
                                      "'매출액총이익률(%)', '경상수지비율', '매입채무회전율(회)', "
                                      "'Closed_In_1Yr']",
                     'Closed_In_2Yrs': "['당기순이익(손실)/자산총계', '순운전자본/자산총계', "
                                       "'매출채권', '비유동자산', '유형자산', '판매비와관

In [8]:
additional_features = {
    '제조업(10~34)' : [
        '3YEAR TREASURY',
        '5YEAR TREASURY',
        '10YEAR TREASURY',
        '3YEAR COPORATE BOND',
        'CD 91DAYS',
        'CALL RATE 1DAY',
        'BASE RATE',
        'DOW_JONES_COMMODITY_INDEX_PRICE',
        'DOW_JONES_COMMODITY_INDEX_CHANGE_RATE',
        'EXCHANGE_RATE_PRICE',
        'EXCHANGE_RATE_CHANGE_RATE',
    ],
    '부동산업(68)' : [
        'NATIONWIDE',
        'CAPITAL AREA',
        'NON CAPITAL AREA'
    ],
    '도매 및 소매업(45~47)' : [
        'CSI',
        'CCI',
        'CLI'
    ],
    '숙박 및 음식점업(55~56)' : [
        'CSI',
        'CCI',
        'CLI'
    ],
    '건설업(41~42)' : [
        '3YEAR TREASURY',
        '5YEAR TREASURY',
        '10YEAR TREASURY',
        '3YEAR COPORATE BOND',
        'CD 91DAYS',
        'CALL RATE 1DAY',
        'BASE RATE',
        'DOW_JONES_COMMODITY_INDEX_PRICE',
        'DOW_JONES_COMMODITY_INDEX_CHANGE_RATE',
        'EXCHANGE_RATE_PRICE',
        'EXCHANGE_RATE_CHANGE_RATE',
    ]
}

# Load data

In [9]:
from data_loader import SectorDataLoader

data_loader = SectorDataLoader()
data_finance = (
    data_loader
    .load_dataset()
    .pipe(data_loader.add_beaver_indicator)
    .pipe(data_loader.add_label)
)
data_finance.head()

,BIZ_NO,CMP_PFIX_NM,CMP_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_TYP,CMP_SCL,PBCO_GB,...,STAT_OCR_DATE_y,Closed_Year,유동자산/부채총계,당기순이익(손실)/자산총계,부채총계/자산총계,순운전자본/자산총계,유동부채/유동자산,Years_From_Closed_Year_To_FS,Closed_In_1Yr,Closed_In_2Yrs
117,1018135422,(주),한국선박기술,NaN,NaN,Korea Marine Time Service,M,NaN,2,2,...,20200930.0,2020.0,1.031451,0.038773,0.578094,0.543045,0.089273,2.0,0,1
219,1018154206,(주),드림미즈,NaN,NaN,"dreammiz Co., Ltd.",M,NaN,2,2,...,20201231.0,2020.0,1.220040,0.001177,0.662356,0.608449,0.247063,2.0,0,1
279,1018163684,NaN,디에프에스서울,NaN,(주),DFS Seoul Ltd.,M,NaN,2,2,...,20190917.0,2019.0,11.012914,0.046666,0.090794,0.909114,0.090802,1.0,1,1
339,1018178760,(주),대교디앤씨,NaN,NaN,"DAEGYO D & C CO.,LTD.",M,NaN,2,2,...,20191010.0,2019.0,0.916364,-0.026128,1.091269,0.097514,0.902486,1.0,1,1
406,1018194173,NaN,에코에너지,NaN,(주),"Eco Energy Co.,Ltd.",M,NaN,2,2,...,20200831.0,2020.0,1.183355,-0.062095,0.828142,0.194625,0.801400,2.0,0,1


In [10]:
from data_loader import IndexDataLoader
import pandas as pd
import ast

index_features = [
     '3YEAR TREASURY',
     '5YEAR TREASURY',
     '10YEAR TREASURY',
     '3YEAR COPORATE BOND',
     'CD 91DAYS',
     'CALL RATE 1DAY',
     'BASE RATE',
     'YEAR_MONTH',
     'DOW_JONES_COMMODITY_INDEX_PRICE',
     'DOW_JONES_COMMODITY_INDEX_CHANGE_RATE',
     'EXCHANGE_RATE_PRICE',
     'EXCHANGE_RATE_CHANGE_RATE',
     'NATIONWIDE',
     'CAPITAL AREA',
     'NON CAPITAL AREA',
     'CSI',
     'CCI',
     'CLI',
]

index_data_loader = IndexDataLoader()

index_data = (
    index_data_loader.load_interest_rate()
    .merge(
        (
            index_data_loader.load_dow_jones_commodity_index()
            .rename(columns={'PRICE':'DOW_JONES_COMMODITY_INDEX_PRICE',
                            'CHANGE_RATE':'DOW_JONES_COMMODITY_INDEX_CHANGE_RATE'})
        ),
        how='outer',
        left_on='YEAR_MONTH',
        right_on='YEAR_MONTH'
    )
    .merge(
        (
            index_data_loader.load_exchange_rate()
            .rename(columns={'PRICE':'EXCHANGE_RATE_PRICE',
                            'CHANGE_RATE':'EXCHANGE_RATE_CHANGE_RATE'})
        ),
        how='outer',
        left_on='YEAR_MONTH',
        right_on='YEAR_MONTH'
    )
    .merge(
        (
            index_data_loader.load_apartment_housing_index()
        ),
        how='outer',
        left_on='YEAR_MONTH',
        right_on='YEAR_MONTH'
        
    )
    .merge(
        (
            index_data_loader.load_csi()
        ),
        how='outer',
        left_on='YEAR_MONTH',
        right_on='YEAR_MONTH'
    )
    .merge(
        (
            index_data_loader.load_composite_index()
            .rename(columns={'PRICE':'DOW_JONES_COMMODITY_INDEX_PRICE',
                            'CHANGE_RATE':'DOW_JONES_COMMODITY_INDEX_CHANGE_RATE'})
        ),
        how='outer',
        left_on='YEAR_MONTH',
        right_on='YEAR_MONTH'
    )
    .assign(year=lambda x : x['YEAR_MONTH'].dt.year)
    .assign(month=lambda x : x['YEAR_MONTH'].dt.month)
    .groupby(['year'])
    [index_features].mean()
    .reset_index(drop=False)
)

index_data

,year,3YEAR TREASURY,5YEAR TREASURY,10YEAR TREASURY,3YEAR COPORATE BOND,CD 91DAYS,CALL RATE 1DAY,BASE RATE,DOW_JONES_COMMODITY_INDEX_PRICE,DOW_JONES_COMMODITY_INDEX_CHANGE_RATE,EXCHANGE_RATE_PRICE,EXCHANGE_RATE_CHANGE_RATE,NATIONWIDE,CAPITAL AREA,NON CAPITAL AREA,CSI,CCI,CLI
0,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.841667,68.600000,58.416667,NaN,NaN,NaN
1,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.758333,82.441667,60.925000,NaN,NaN,NaN
2,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.775000,89.150000,62.991667,NaN,NaN,NaN
3,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.058333,87.833333,64.200000,NaN,NaN,NaN
4,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.733333,86.833333,69.516667,NaN,NaN,NaN
5,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.700000,86.850000,80.983333,NaN,NaN,NaN
6,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.883333,81.891667,85.250000,NaN,NaN,NaN
7,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84.325000,80.333333,87.125000,NaN,NaN,NaN
8,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.008333,83.366667,91.325000,NaN,NaN,NaN
9,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.008333,89.175000,97.658333,NaN,NaN,NaN


In [11]:
data = (
    data_finance
    .assign(year=lambda x : x['결산년월'].astype(str).str[:4].astype(int))
    .merge(
        index_data,
        how='left',
        left_on='year',
        right_on='year'
    )
)
data

,BIZ_NO,CMP_PFIX_NM,CMP_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_TYP,CMP_SCL,PBCO_GB,...,DOW_JONES_COMMODITY_INDEX_PRICE,DOW_JONES_COMMODITY_INDEX_CHANGE_RATE,EXCHANGE_RATE_PRICE,EXCHANGE_RATE_CHANGE_RATE,NATIONWIDE,CAPITAL AREA,NON CAPITAL AREA,CSI,CCI,CLI
0,1018135422,(주),한국선박기술,NaN,NaN,Korea Marine Time Service,M,NaN,2,2,...,625.940833,-0.695000,1098.985000,0.372500,100.608333,104.325000,97.441667,104.083333,101.425000,100.058333
1,1018154206,(주),드림미즈,NaN,NaN,"dreammiz Co., Ltd.",M,NaN,2,2,...,625.940833,-0.695000,1098.985000,0.372500,100.608333,104.325000,97.441667,104.083333,101.425000,100.058333
2,1018163684,NaN,디에프에스서울,NaN,(주),DFS Seoul Ltd.,M,NaN,2,2,...,625.940833,-0.695000,1098.985000,0.372500,100.608333,104.325000,97.441667,104.083333,101.425000,100.058333
3,1018178760,(주),대교디앤씨,NaN,NaN,"DAEGYO D & C CO.,LTD.",M,NaN,2,2,...,625.940833,-0.695000,1098.985000,0.372500,100.608333,104.325000,97.441667,104.083333,101.425000,100.058333
4,1018194173,NaN,에코에너지,NaN,(주),"Eco Energy Co.,Ltd.",M,NaN,2,2,...,625.940833,-0.695000,1098.985000,0.372500,100.608333,104.325000,97.441667,104.083333,101.425000,100.058333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10516,8998700283,(주),덕성건설,NaN,NaN,Deokseong Construction,M,NaN,2,2,...,625.940833,-0.695000,1098.985000,0.372500,100.608333,104.325000,97.441667,104.083333,101.425000,100.058333
10517,8998700283,(주),덕성건설,NaN,NaN,Deokseong Construction,M,NaN,2,2,...,614.237500,0.997500,1164.787500,0.317500,100.108333,107.183333,94.075000,99.083333,100.258333,98.475000
10518,8998701263,NaN,하나머스트제6호기업인수목적,NaN,(주),HANA MUST SIXTH SPECIAL PURPOSE ACQUISITION CO...,M,NaN,2,2,...,625.940833,-0.695000,1098.985000,0.372500,100.608333,104.325000,97.441667,104.083333,101.425000,100.058333
10519,8998701263,NaN,하나머스트제6호기업인수목적,NaN,(주),HANA MUST SIXTH SPECIAL PURPOSE ACQUISITION CO...,M,NaN,2,2,...,614.237500,0.997500,1164.787500,0.317500,100.108333,107.183333,94.075000,99.083333,100.258333,98.475000


# Baseline classifier using Beaver's features

In [12]:
from project_paths import DATA_PATH

sectors = [
    '제조업(10~34)',
    '부동산업(68)',
    '도매 및 소매업(45~47)',
    '숙박 및 음식점업(55~56)',
    '건설업(41~42)'
]

beaver_features = [
    '유동자산/부채총계',
    '당기순이익(손실)/자산총계',
    '부채총계/자산총계',
    '순운전자본/자산총계',
    '유동부채/유동자산'
]

extended_financial_features = [
    '유동자산',
    '매출채권',
    '비유동자산',
    '유형자산',
    '자산총계',
    '유동부채',
    '비유동부채',
    '부  채  총  계',
    '자본금',
    '이익잉여금(결손금）',
    '자본총계',
    '매출액',
    '판매비와관리비',
    '영업이익（손실）',
    '법인세비용차감전순손익',
    '법인세비용',
    '당기순이익(손실)',
    '기업순이익률(%)',
    '유보액/총자산(%)',
    '유보액/납입자본(%)',
    '매출액총이익률(%)',
    '매출액영업이익률(%)',
    '매출액순이익률(%)',
    '수지비율(%)',
    '경상수지비율',
    '영업비율(%)',
    '금융비용대매출액비율(%',
    '금융비용대부채비율(%)',
    '금융비용대총비용비율(%',
    '부채비율(%)',
    '차입금의존도(%)',
    '자기자본비율(%)',
    '순운전자본비율(%)',
    '유동부채비율(%)',
    '비유동부채비율(%)',
    '부채총계대 매출액(%)',
    '총자본회전율(회)',
    '재고자산회전율(회)',
    '매출채권회전율(회)',
    '매입채무회전율(회)',
    '미수금',
    '매출원가',
    '무형자산',
    '재고자산',
]

years_to_close = [
    'Closed_In_1Yr',
    'Closed_In_2Yrs',
]

x_data_year = list(range(2018, 2020))

EXP_RESULT_PATH = DATA_PATH / 'experiment_result' / exp_name

if not os.path.exists(EXP_RESULT_PATH) : 
    os.mkdir(EXP_RESULT_PATH)

In [13]:
import numpy as np
import pandas as pd
from pycaret.classification import ClassificationExperiment
from tqdm import tqdm

In [14]:
exp_result = None

for target_years_to_close in tqdm(years_to_close) : 
    for x_data_target_year in tqdm(x_data_year, leave=True) : 
        for sector in sectors : 

            filename = EXP_RESULT_PATH / f'Beaver_baseline_{sector}_{target_years_to_close}_{x_data_target_year}.csv'

            if not os.path.exists(filename) : 

                data_to_train = (
                    data
                    .loc[~data[beaver_features[0]].isin([np.nan, np.inf, -np.inf]), :]
                    .loc[~data[beaver_features[1]].isin([np.nan, np.inf, -np.inf]), :]
                    .loc[~data[beaver_features[2]].isin([np.nan, np.inf, -np.inf]), :]
                    .loc[~data[beaver_features[3]].isin([np.nan, np.inf, -np.inf]), :]
                    .loc[~data[beaver_features[4]].isin([np.nan, np.inf, -np.inf]), :]
                    .loc[lambda x : pd.to_datetime(x['결산년월'], format='%Y%m%d').dt.year==x_data_target_year]
                    .loc[lambda x : x['대분류']==sector]
                )

                exp = ClassificationExperiment()

                target_features = (
                    [x for x in ast.literal_eval(best_features[sector][target_years_to_close]) if 'Closed' not in x]
                    + index_data.drop(['year'], axis=1).columns.tolist() #+ additional_features[sector]
                    + [target_years_to_close]
                )
                              
                exp.setup(
                    (
                        data_to_train
                        .loc[:, target_features]
                        .reset_index(drop=True)
                        .fillna(0)
                    ),
                    target=target_years_to_close,
                    #feature_selection=True,
                    #remove_multicollinearity = True,
                )

                models = exp.compare_models(sort='AUC', n_select=3)
                try : 
                    tuned_models = [exp.tune_model(x) for x in models]
                except : 
                    tuned_models = models
                
                #if target_years_to_close == 'Closed_In_1Yr' :
                blended_model = exp.blend_models(estimator_list=tuned_models, fold=5, method = 'soft')
                result = (
                    exp.pull()
                    .assign(target=target_years_to_close)
                    .assign(x_data_yaer=x_data_target_year)
                    .assign(대분류=sector)
                    .reset_index(drop=False)
                    .rename(columns={'index':'model_name'})
                )
                result['feature_selection'] = str(exp.dataset_transformed.columns.tolist())
                result.to_csv(filename)

            else : 
                result = pd.read_csv(filename)

            if exp_result is None : 
                exp_result = result
            else : 
                exp_result = pd.concat([exp_result, result], axis=0)

  0%|                                                                                                                                                                                                                   | 0/2 [00:00<?, ?it/s]

,Description,Value
0,Session id,3884
1,Target,Closed_In_1Yr
2,Target type,Binary
3,Original data shape,"(1800, 28)"
4,Transformed data shape,"(1800, 11)"
5,Transformed train set shape,"(1259, 11)"
6,Transformed test set shape,"(541, 11)"
7,Numeric features,27
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8745,0.6682,0.1471,0.5598,0.2250,0.1845,0.2331,0.1240
gbc,Gradient Boosting Classifier,0.8689,0.6480,0.1037,0.5107,0.1614,0.1255,0.1753,0.0690
lightgbm,Light Gradient Boosting Machine,0.8753,0.6437,0.1772,0.5785,0.2568,0.2125,0.2586,0.0560
et,Extra Trees Classifier,0.8705,0.6428,0.0982,0.5517,0.1594,0.1253,0.1811,0.0890
ada,Ada Boost Classifier,0.8642,0.6285,0.0919,0.4117,0.1481,0.1060,0.1423,0.0340
nb,Naive Bayes,0.8650,0.6117,0.0982,0.3171,0.1482,0.1089,0.1308,0.0110
qda,Quadratic Discriminant Analysis,0.3184,0.5991,0.7816,0.1220,0.1975,0.0088,0.0433,0.0170
lda,Linear Discriminant Analysis,0.8674,0.5986,0.0371,0.3167,0.0655,0.0457,0.0772,0.0160
knn,K Neighbors Classifier,0.8547,0.5829,0.0743,0.2808,0.1130,0.0646,0.0820,0.0240
dt,Decision Tree Classifier,0.7697,0.5494,0.2261,0.1709,0.1909,0.0605,0.0633,0.0220


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8571,0.6216,0.2500,0.4000,0.3077,0.2327,0.2408
1,0.9048,0.5977,0.3750,0.7500,0.5000,0.4538,0.4872
2,0.8889,0.5489,0.1250,1.0000,0.2222,0.1996,0.3330
3,0.8810,0.6653,0.1250,0.6667,0.2105,0.1775,0.2531
4,0.8730,0.7219,0.1875,0.5000,0.2727,0.2186,0.2505
5,0.8651,0.5677,0.0588,0.5000,0.1053,0.0791,0.1357
6,0.8730,0.6425,0.1765,0.6000,0.2727,0.2252,0.2767
7,0.8651,0.6881,0.1176,0.5000,0.1905,0.1466,0.1935
8,0.8571,0.6794,0.1176,0.4000,0.1818,0.1284,0.1577


Fitting 10 folds for each of 10 candidates, totalling 100 fits


...

Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8571,0.8667,1.0000,0.8333,0.9091,0.5882,0.6455
1,0.7143,0.8000,0.7333,0.8462,0.7857,0.3636,0.3721
2,0.7143,0.4778,0.8667,0.7647,0.8125,0.2222,0.2301
3,0.8571,0.7222,1.0000,0.8333,0.9091,0.5882,0.6455
4,0.7143,0.4889,0.8667,0.7647,0.8125,0.2222,0.2301
5,0.7000,0.6000,0.8667,0.7647,0.8125,0.0769,0.0808
6,0.7500,0.6933,0.9333,0.7778,0.8485,0.1667,0.1925
7,0.7500,0.7467,0.8000,0.8571,0.8276,0.3750,0.3780
8,0.8000,0.7067,0.8000,0.9231,0.8571,0.5294,0.5447


Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7561,0.7515,0.8333,0.8333,0.8333,0.3788,0.3788
1,0.7805,0.6545,0.9333,0.8000,0.8615,0.3469,0.3723
2,0.7317,0.6576,0.8667,0.7879,0.8254,0.2521,0.2575
3,0.6829,0.6394,0.8667,0.7429,0.8000,0.0566,0.0608
4,0.7561,0.7443,0.8621,0.8065,0.8333,0.3807,0.3836
Mean,0.7415,0.6895,0.8724,0.7941,0.8307,0.2830,0.2906
Std,0.0331,0.0482,0.0329,0.0296,0.0197,0.1225,0.1241



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [09:10<00:00, 275.09s/it]


In [15]:
aggregated_result_path = EXP_RESULT_PATH / 'result.csv'

if not os.path.exists(aggregated_result_path) :
    exp_result.to_csv(aggregated_result_path)
    
exp_result

,Fold,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,target,x_data_yaer,대분류,feature_selection
0,0,0.8810,0.6401,0.1515,0.7143,0.2500,0.2140,0.2923,Closed_In_1Yr,2018,제조업(10~34),"['순운전자본/자산총계', '매출채권', '비유동자산', '유형자산', '자본금',..."
1,1,0.8810,0.6276,0.0909,1.0000,0.1667,0.1481,0.2828,Closed_In_1Yr,2018,제조업(10~34),"['순운전자본/자산총계', '매출채권', '비유동자산', '유형자산', '자본금',..."
2,2,0.8651,0.6541,0.1515,0.4545,0.2273,0.1731,0.2049,Closed_In_1Yr,2018,제조업(10~34),"['순운전자본/자산총계', '매출채권', '비유동자산', '유형자산', '자본금',..."
3,3,0.8849,0.6905,0.2121,0.7000,0.3256,0.2818,0.3429,Closed_In_1Yr,2018,제조업(10~34),"['순운전자본/자산총계', '매출채권', '비유동자산', '유형자산', '자본금',..."
4,4,0.8765,0.6652,0.2188,0.5385,0.3111,0.2563,0.2880,Closed_In_1Yr,2018,제조업(10~34),"['순운전자본/자산총계', '매출채권', '비유동자산', '유형자산', '자본금',..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2,2,0.7317,0.6576,0.8667,0.7879,0.8254,0.2521,0.2575,Closed_In_2Yrs,2019,건설업(41~42),"['매출채권', '비유동자산', '유형자산', '판매비와관리비', '기업순이익률(%..."
3,3,0.6829,0.6394,0.8667,0.7429,0.8000,0.0566,0.0608,Closed_In_2Yrs,2019,건설업(41~42),"['매출채권', '비유동자산', '유형자산', '판매비와관리비', '기업순이익률(%..."
4,4,0.7561,0.7443,0.8621,0.8065,0.8333,0.3807,0.3836,Closed_In_2Yrs,2019,건설업(41~42),"['매출채권', '비유동자산', '유형자산', '판매비와관리비', '기업순이익률(%..."
5,Mean,0.7415,0.6895,0.8724,0.7941,0.8307,0.2830,0.2906,Closed_In_2Yrs,2019,건설업(41~42),"['매출채권', '비유동자산', '유형자산', '판매비와관리비', '기업순이익률(%..."
